### Plot shape difference between XX, YY measured moments and PSF XX, YY moments

Adpated by Michael Wood-Vasey from Merlin Fisher-Levine on 2018-08-21.



In [8]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import lsst.daf.persistence as dafPersist
import lsst.afw.image as afwImage
%matplotlib inline

In [9]:
def plot(mags, diffs, title='', saveFilename=''):
    maxDiff = 12
    nStdev = 3.0

    ptSize = 25
    fontsize = 20
    color = 'b'

    left, width = 0.12, 0.62
    bottom, height = 0.10, 0.85 # had to change height to make it save correctly
    left_h = left + width + 0.03
    bottom_h = bottom + width + 0.02
    rect_scatter = [left, bottom, width, height]
    rect_histy = [left_h, bottom, 0.20, height]

    deltaMin = 0.0

    fig = plt.figure(figsize=(15,10))
    axScatter = plt.axes(rect_scatter)
    if title: plt.title(title, fontsize=25)
    axHisty = plt.axes(rect_histy)

    _ = axScatter.scatter(mags, diffs, s=1.3*ptSize, marker="o", facecolors="b", edgecolors=color, alpha = 0.15)
    _ = axScatter.axhline(0, ls='--', color="0.4")

    axScatterY1 = axScatter.get_ylim()[0]
    axScatterY2 = axScatter.get_ylim()[1]
    nyDecimal = int(-1.0*np.around(np.log10(0.05*abs(axScatterY2 - (axScatterY1 - deltaMin))) - 0.5))
    yBinwidth = max(0.5/10**nyDecimal, np.around(0.02*abs(axScatterY2 - (axScatterY1 - deltaMin)), nyDecimal))

    yBins = np.arange((axScatterY1 - deltaMin) - 0.5*yBinwidth, axScatterY2 + 0.55*yBinwidth, yBinwidth)
    _ = axHisty.hist(diffs, bins=yBins, color='b', alpha=0.6, orientation="horizontal", label='test')

    _ = axScatter.set_ylim(-1.*maxDiff,maxDiff)
    _ = axHisty.set_ylim(-1.*maxDiff,maxDiff)
    axHisty.set_xscale("log", nonposy="clip")
    
    axScatter.set_xlabel('mag')
    axScatter.set_ylabel('Percentage difference star - model size (sqrt(<xx+yy>/2)')
    
    mean = np.mean(diffs)
    std = np.std(diffs)
    std_clip = np.std( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    mean_clip = np.mean ( np.clip(diffs, mean-nStdev*std, mean+nStdev*std))
    
    spc = 0.04
    _ = plt.text(0.05, 0.95-0*spc, "Star N       = %s"  %len(diffs), ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-1*spc, "stddev      = %.4f" %std,        ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-2*spc, "std_clip     = %.4f"%std_clip,   ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-3*spc, "mean        = %.4f" %mean,       ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')
    _ = plt.text(0.05, 0.95-4*spc, "mean_clip = %.4f"   %mean_clip,  ha="left", va="center", fontsize=fontsize, transform=axScatter.transAxes, color='k')

    if saveFilename:
        plt.savefig(saveFilename)


In [35]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
outdir = './plots'

try:
    os.makedirs(outdir)
except:
    pass

In [28]:
butler = dafPersist.Butler(repo)
run = os.path.basename(os.path.dirname(repo))
print(run)

Run1.1


In [37]:
visits = [204560, 204661, 211181, 211182, 211240, 211274, 263508, 269133, 428497, 428530, 437349]
rafts = ['{:1d},{:1d}'.format(x, y) for x in range(5) for y in range(5)]
sensors = ['{:1d},{:1d}'.format(x, y) for x in range(3) for y in range(3)]

corners = ['0,0', '0,4', '4,4', '4,0']
for corner in corners:
    rafts.remove(corner)

In [ ]:
print(rafts)

In [38]:
from copy import copy

def get_shape_mag_diff(visits, partial_data_id):
    star_sizes = []
    model_sizes = []
    mags = []
    for visit in visits:
        data_id = copy(partial_data_id)
        data_id['visit'] = visit
        try:
            cat = butler.get('src', dataId=data_id)  
            calib = butler.get('calexp_calib', dataId=data_id)
        except dafPersist.NoResults as e:
            print(e)
            continue

        calib.setThrowOnNegativeFlux(False)

        mask = (cat['calib_psfUsed'] == 1) #& (cat['base_PixelFlags_flag_interpolated'] == 0)
        cat = cat[mask]
        star_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_xx'] + cat['base_SdssShape_yy']))])
        model_sizes.extend([_ for _ in np.sqrt( 0.5*(cat['base_SdssShape_psf_xx'] + cat['base_SdssShape_psf_yy']))])
        mags.extend([_ for _ in calib.getMagnitude(cat['base_PsfFlux_flux'])])
        
    diffs = [100*(a - b)/b for a,b in zip(star_sizes, model_sizes)]
    diffs = np.asarray(diffs)

    return mags, diffs



In [ ]:
for raft in rafts:
    for sensor in sensors:
        partial_data_id = {'raft': raft, 'sensor': sensor}
        mags, diffs = get_shape_mag_diff(visits, partial_data_id)

        if len(mags) < 1:
            print('No results for {}'.format(partial_data_id))
            continue
    
        title = '{}_raft_{}_sensor_{}'.format(run, raft, sensor)
        saveName = os.path.join(outdir, title + '.png')
        plot(mags, diffs, title, saveName)  
        print('Finished raft, sensor: {}, {}'.format(raft, sensor))
        sys.stdout.flush()    

In [22]:
fails = []
for raft in rafts:
    for sensor in sensors:
        for visit in visits:
            try:
                cat = butler.get('src', dataId={'raft': raft, 'sensor': sensor, 'visit': visit})
            except:
                fails.append((raft, sensor, visit))

In [23]:
fails

[('1,1', '2,0', 263508)]

In [ ]:
vis = '1228..1232:2^1238..1248:2^19658..19662:2^19680..19684:2^19694..19698:2^19708..19712:2^30482..30504:2'


In [ ]:
# """Parse ``--id`` data and append results to ``namespace.<argument>.idList``.
# Parameters
# ----------
# parser : `ArgumentParser`
#     Argument parser.
# namespace : `argparse.Namespace`
#     Parsed command (an instance of argparse.Namespace). The following attributes are updated:
#     - ``<idName>.idList``, where ``<idName>`` is the name of the ID argument, for instance ``"id"``
#       for ID argument ``--id``.
# values : `list`
#     A list of data IDs; see Notes below.
# option_string : `str`
#     Option value specified by the user.
# Notes
# -----
# The data format is::
#     key1=value1_1[^value1_2[^value1_3...] key2=value2_1[^value2_2[^value2_3...]...
# The values (e.g. ``value1_1``) may either be a string, or of the form ``"int..int"``
# (e.g. ``"1..3"``) which is interpreted as ``"1^2^3"`` (inclusive, unlike a python range).
# So ``"0^2..4^7..9"`` is equivalent to ``"0^2^3^4^7^8^9"``.  You may also specify a stride:
# ``"1..5:2"`` is ``"1^3^5"``.
# The cross product is computed for keys with multiple values. For example::
#     --id visit 1^2 ccd 1,1^2,2
# results in the following data ID dicts being appended to ``namespace.<argument>.idList``:
#     {"visit":1, "ccd":"1,1"}
#     {"visit":2, "ccd":"1,1"}
#     {"visit":1, "ccd":"2,2"}
#     {"visit":2, "ccd":"2,2"}
# """
def parse(input_string):
    import collections, re    
    
    idDict = collections.OrderedDict()
    name = 'visit'
#     for nameValue in values:
#         name, sep, valueStr = nameValue.partition("=")
#         if name in idDict:
#             parser.error("%s appears multiple times in one ID argument: %s" % (name, option_string))
    idDict[name] = []
    for v in input_string.split("^"):
        mat = re.search(r"^(\d+)\.\.(\d+)(?::(\d+))?$", v)
        if mat:
            v1 = int(mat.group(1))
            v2 = int(mat.group(2))
            v3 = mat.group(3)
            v3 = int(v3) if v3 else 1
            for v in range(v1, v2 + 1, v3):
                idDict[name].append(str(v))
        else:
            idDict[name].append(v)

# iterList = [idDict[key] for key in idDict.keys()]
# idDictList = [collections.OrderedDict(zip(idDict.keys(), valList))
#               for valList in itertools.product(*iterList)]

# argName = option_string.lstrip("-")
# ident = getattr(namespace, argName)
# ident.idList += idDictList
    return idDict

In [ ]:
parse(vis)

In [ ]:
ptSize = 25
color = 'b'
fig = plt.figure(figsize=(15,10))
axScatter = plt.axes()

axScatter.axhline(0, ls='--', color="0.4")
axScatter.scatter(mag, diffs, s=1.3*ptSize, marker="o", facecolors="b", edgecolors=color, alpha = 0.2)
# axScatter.axhline(np.mean(diffs), ls='--', color="0.4")
plt.ylim(-8,8)